# 0. 데이터 로드 

In [12]:
from datasets import load_dataset
dataset = load_dataset("Cartinoe5930/web_text_synthetic_dataset_50k", split = "train")

## 1. 역색인

In [5]:
from kiwipiepy import Kiwi
from collections import defaultdict
import math

In [19]:
class InvertedIndex:
    def __init__(self):
        self.reset()

    def reset(self):
        self.index = defaultdict(dict)
        self.kiwi = Kiwi()
        self.document_lengths = {}
        self.total_documents = 0
        self.average_document_length = 0
        self.documents = {}

    def tokenize(self, text):
        return [token.form for token in self.kiwi.tokenize(text)]

    def add_document(self, doc_id, question, answer, content='None'):
        tokens = self.tokenize(question)
        self.document_lengths[doc_id] = len(tokens)
        self.total_documents += 1
        self.documents[doc_id] = {'question': question, 'meta': {'answer': answer, 'content':content}}

        for token in set(tokens):
            if doc_id not in self.index[token]:
                self.index[token][doc_id] = 0
            self.index[token][doc_id] += tokens.count(token)

        self.average_document_length = sum(self.document_lengths.values()) / self.total_documents

    def calculate_bm25_score(self, query_tokens, doc_id):
        k1 = 1.5
        b = 0.75
        score = 0

        for token in query_tokens:
            if token not in self.index or doc_id not in self.index[token]:
                continue

            tf = self.index[token][doc_id]
            df = len(self.index[token])
            idf = math.log((self.total_documents - df + 0.5) / (df + 0.5) + 1)

            numerator = tf * (k1 + 1)
            denominator = tf + k1 * (1 - b + b * self.document_lengths[doc_id] / self.average_document_length)
            score += idf * numerator / denominator

        return score

    def search(self, query, k=5):
        query_tokens = self.tokenize(query)
        scores = defaultdict(float)

        for token in query_tokens:
            if token in self.index:
                for doc_id in self.index[token]:
                    scores[doc_id] += self.calculate_bm25_score(query_tokens, doc_id)

        top_k = sorted(scores.items(), key=lambda x: x[1], reverse = True)[:k]
        return [(doc_id, score, self.documents[doc_id]) for doc_id, score in top_k]

In [23]:
# add docs
from tqdm import tqdm

inverted_index = InvertedIndex()
for idx, data in enumerate(tqdm(dataset)):
    question = data['question']
    answer = data['response']
    inverted_index.add_document(idx, question, answer)

100%|██████████| 21950/21950 [02:15<00:00, 162.15it/s]


# 2. 질문 마다 10개 유사한 질문 뽑기

In [60]:
temp_question = dataset[2]['question']
temp_answer = dataset[2]['response']
print(temp)
print("-" * 100)
print(temp_answer)


GBP 수익률 곡선을 USD OIS 금리 곡선과 FX 스왑 환율을 사용하여 QuantLib을 이용해 계산하고자 합니다. 다음 중 어떤 단계를 거쳐야 할까요?

1. USD OIS 스왑 금리를 사용하여 OIS 곡선을 부트스트랩하고, 이를 통해 얻은 할인 계수를 확인하여 올바른 값이 나오는지 체크하였습니다. 
   
2. 이후 GBP 수익률 곡선을 부트스트랩하기 위해 FX 스왑 레이트 헬퍼를 설정하고, 해당 헬퍼들이 올바르게 구성되었는지 확인합니다.

3. 마지막으로, 다른 라이브러리와 비교하여 얻은 할인 계수가 다를 경우, 어떤 요인들로 인해 차이가 발생할 수 있는지 설명하고, 이를 해결하기 위한 방안을 제시해주세요.

이를 통해 수익률 곡선의 신뢰성을 높이고, 향후 금융 상품 가격 책정 시 발생할 수 있는 오류를 최소화할 수 있도록 합니다.
----------------------------------------------------------------------------------------------------
AR(1) 과정이나 Ornstein-Uhlenbeck 과정에서 반감기를 계산하는 과정은 매우 중요한데, 특히 더 많은 회귀 변수를 고려할 때 그 결과가 어떻게 달라지는지를 이해하는 것이 필요합니다.

### 기본적인 반감기 계산

주어진 회귀 모델을 통해 피팅한 결과 `res`에서 반감기를 계산하기 위한 방정식은 다음과 같습니다:

\[
\text{halflife} = \text{round}(-\text{np.log}(0.5) / \text{res.params}[1], 0)
\]

여기에서 `res.params[1]`은 모델의 파라미터 중 하나로, 이 값이 정확히 평균 회귀 속도를 반영하는 것이어야 합니다. 그러나 만약 회귀 변수(exog)가 추가되면, 이 값의 해석이 복잡해질 수 있습니다. 

### 추가 회귀 변수를 포함할 경우

추가적인 회귀 변수가 포함된 경우, 평균 되돌림 과정이 다음과 같은 두 가지 관계를 생성합니다:

\[
h - g

In [26]:
result = inverted_index.search(temp, k=10)
result

[(0,
  162012.3380894895,
  {'question': '주식이 배당금을 지급하지 않는 경우, 미국형 콜 옵션의 가격을 평가하는 방법에 대해 설명하시오. 구체적으로 단순한 이항모형을 사용하며, 위험 중립 가정을 적용하는 방법을 서술하시오. 다음 질문에 답하시오:\n\n1. 유럽형 콜 옵션의 가격이 이항모형에서 어떻게 계산되는지 설명하시오.\n2. 미국형 콜 옵션에 대한 조기 실행이 최적이 아닌 이유를 논하시오.\n3. 배당금을 지급하지 않는 기초 자산에 대해 유럽형 콜 옵션과 미국형 콜 옵션이 동일하다고 하는 주장의 의미는 무엇인지 설명하시오.\n4. 깊게 인더 머니(deep in-the-money)인 미국형 콜 옵션의 가격을 어떻게 결정할 수 있는지, 그리고 왜 옵션 가격이 (1)식 대신 (2)식으로 계산되어야 하는지를 설명하시오. \n\n위 질문에 대한 답변을 작성하시오.',
   'meta': {'answer': '1. **유럽형 콜 옵션의 가격 계산**:\n   이항모형에서 유럽형 콜 옵션의 가격은 주식 가격이 여러 시점에 걸쳐 두 가지 가능한 값 중 하나로 변할 수 있는 구조를 기반으로 한다. 이항 구조에서는 특정 시간 t에 주가가 상승할 확률 p와 하락할 확률 (1-p)을 설정하고, 각 기초 자산의 가격 변수를 다음과 같이 정의한다:\n   - S_up = S * u (주가 상승)\n   - S_down = S * d (주가 하락)\n   여기서 u는 상승 비율, d는 하락 비율이다. 옵션의 만기 시점에서의 가치 C(T) = max(0, S(T) - K)로 정의되며, 이 값을 기초 자산의 변화에 따라 이산적으로 계산한다. 위험 중립적 확률을 사용해 현재 가치를 계산하고, 최종 시점의 옵션 가치의 가중치를 합산하여 현재 시점에서의 옵션 가격을 얻는다.\n\n2. **미국형 콜 옵션의 조기 실행**:\n   배당금을 지급하지 않는 경우, 미국형 콜 옵션을 조기 실행하는 것이 최적이 아닌 이유는 옵션의 시간 가치 때문이다. 옵션을 만기 

## 3. 오답 생성하기

In [50]:
import openai
from dotenv import load_dotenv
import os

dotenv_path = os.path.join(os.getcwd(),'KRX', '.env')
load_dotenv(dotenv_path)

openai_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI(api_key=openai_key)


In [62]:
instruction = "You will given a question, gold answer and one irrelevant answer. Your job is to transform the irrelvant answer to a well-designed wrong answer. Replace mentions of different companies to the company of interest in the question. After the detail a bit to make good wrong answer. please write in korean"

prompt_template = """
### Question: {}
### Gold Answer: {}
### irrelevant Answer: {}
### Good Wrong Answer:
"""

In [63]:
# 9.27 -> 8.94 => 0.33 달러 ㅎㄷㄷ
question = dataset[0]['question']
gold_answer = dataset[0]['response']

selections = []

for item in result:
    irrelevant_answer = item[2]['meta']['answer']
    prompt = prompt_template.format(question, gold_answer, irrelevant_answer)

    instruction_msg = {"role":"system", "content": instruction}
    user_msg = {"role":'user', "content":prompt}

    response = client.chat.completions.create(
        model='gpt-4-turbo',
        messages = [instruction_msg, user_msg]
    )

    selections.append(response.choices[0].message.content)

selections

['1. **유럽형 콜 옵션의 가격 계산**:\n   이항모형에서 유럽형 콜 옵션의 가격은 주식 가격이 여러 시점에 걸쳐 두 가지 가능한 값 중 하나로 변할 수 있는 구조를 기반으로 한다. 이항 구조에서는 특정 시간 t에 주가가 상승할 확률 p와 하락할 확률 (1-p)을 설정하고, 각 기초 자산의 가격 변수를 다음과 같이 정의한다:\n   - S_up = S * u (주가 상승)\n   - S_down = S * d (주가 하락)\n   여기서 u는 상승 비율, d는 하락 비율이다. 옵션의 만기 시점에서의 가치 C(T) = max(0, S(T) - K)로 정의되며, 이 값을 기초 자산의 변화에 따라 이산적으로 계산한다. 위험 중립적 확률을 사용해 현재 가치를 계산하고, 최종 시점의 옵션 가치의 가중치를 합산하여 현재 시점에서의 옵션 가격을 얻는다.\n\n2. **미국형 콜 옵션의 조기 실행**:\n   배당금을 지급하지 않는 경우, 미국형 콜 옵션을 조기 실행하는 것이 최적이 아닌 이유는 옵션의 시간 가치 때문이다. 옵션을 만기 전 조기 실행하면, 이 옵션이 만기까지 가지고 있을 수 있는 추가적인 시간 가치를 포기하게 된다. 주식이 배당금을 지급하지 않으므로 조기 실행의 이점이 없어, 만기까지 기다려 옵션의 가치를 극대화하는 것이 최선이다.\n\n3. **유럽형 콜 옵션과 미국형 콜 옵션의 동일성**:\n   배당금을 지급하지 않는 기초 자산에 대해 유럽형 콜 옵션과 미국형 콜 옵션이 동일하다는 주장은 두 옵션의 가격이 같다는 의미이다. 배당금이 없을 경우, 두 옵션 모두 동일한 가격으로 결제되어 만기까지의 시장 조건에 따라 가치가 결정되고, 조기 실행의 이점이 없기 때문에 두 옵션의 가격 차이가 발생하지 않는다. 즉, 미국형 콜 옵션은 유럽형의 가격과 같고 추가 비용도 발생하지 않는다.\n\n4. **깊게 인더 머니인 미국형 콜 옵션의 가격 결정**:\n   깊게 인더 머니인 미국형 콜 옵션의 가격을 결정하는 방법은 기본적으로 해당 옵션의 내재 가치를 반영한

In [65]:
print(question)
print()
for idx, item in enumerate(selections):
    print(idx+1, end='')
    print("-" * 100)
    print(item)

주식이 배당금을 지급하지 않는 경우, 미국형 콜 옵션의 가격을 평가하는 방법에 대해 설명하시오. 구체적으로 단순한 이항모형을 사용하며, 위험 중립 가정을 적용하는 방법을 서술하시오. 다음 질문에 답하시오:

1. 유럽형 콜 옵션의 가격이 이항모형에서 어떻게 계산되는지 설명하시오.
2. 미국형 콜 옵션에 대한 조기 실행이 최적이 아닌 이유를 논하시오.
3. 배당금을 지급하지 않는 기초 자산에 대해 유럽형 콜 옵션과 미국형 콜 옵션이 동일하다고 하는 주장의 의미는 무엇인지 설명하시오.
4. 깊게 인더 머니(deep in-the-money)인 미국형 콜 옵션의 가격을 어떻게 결정할 수 있는지, 그리고 왜 옵션 가격이 (1)식 대신 (2)식으로 계산되어야 하는지를 설명하시오. 

위 질문에 대한 답변을 작성하시오.

1----------------------------------------------------------------------------------------------------
1. **유럽형 콜 옵션의 가격 계산**:
   이항모형에서 유럽형 콜 옵션의 가격은 주식 가격이 여러 시점에 걸쳐 두 가지 가능한 값 중 하나로 변할 수 있는 구조를 기반으로 한다. 이항 구조에서는 특정 시간 t에 주가가 상승할 확률 p와 하락할 확률 (1-p)을 설정하고, 각 기초 자산의 가격 변수를 다음과 같이 정의한다:
   - S_up = S * u (주가 상승)
   - S_down = S * d (주가 하락)
   여기서 u는 상승 비율, d는 하락 비율이다. 옵션의 만기 시점에서의 가치 C(T) = max(0, S(T) - K)로 정의되며, 이 값을 기초 자산의 변화에 따라 이산적으로 계산한다. 위험 중립적 확률을 사용해 현재 가치를 계산하고, 최종 시점의 옵션 가치의 가중치를 합산하여 현재 시점에서의 옵션 가격을 얻는다.

2. **미국형 콜 옵션의 조기 실행**:
   배당금을 지급하지 않는 경우, 미국형 콜 옵션을 조기 실행하는 것이 최적이 아닌 이유는 옵

In [66]:
import numpy as np

def filter_selections_by_length(selections, std_dev_threshold=2):
    lengths = [len(sentence) for sentence in selections]
    
    # 평균과 표준편차 계산
    mean_length = np.mean(lengths)
    std_dev_length = np.std(lengths)
    
    # 허용 범위 계산
    lower_bound = mean_length - (std_dev_threshold * std_dev_length)
    upper_bound = mean_length + (std_dev_threshold * std_dev_length)
    
    # 허용 범위 내의 문장만 필터링
    filtered_selections = [sentence for sentence, length in zip(selections, lengths) 
                           if lower_bound <= length <= upper_bound]
    
    return filtered_selections

In [67]:
filtered_selections = filter_selections_by_length(selections)

for idx, select in enumerate(filtered_selections):
    print(idx+1, '-'*100)
    print(select)

1 ----------------------------------------------------------------------------------------------------
1. **유럽형 콜 옵션의 가격 계산**:
   이항모형에서 유럽형 콜 옵션의 가격은 주식 가격이 여러 시점에 걸쳐 두 가지 가능한 값 중 하나로 변할 수 있는 구조를 기반으로 한다. 이항 구조에서는 특정 시간 t에 주가가 상승할 확률 p와 하락할 확률 (1-p)을 설정하고, 각 기초 자산의 가격 변수를 다음과 같이 정의한다:
   - S_up = S * u (주가 상승)
   - S_down = S * d (주가 하락)
   여기서 u는 상승 비율, d는 하락 비율이다. 옵션의 만기 시점에서의 가치 C(T) = max(0, S(T) - K)로 정의되며, 이 값을 기초 자산의 변화에 따라 이산적으로 계산한다. 위험 중립적 확률을 사용해 현재 가치를 계산하고, 최종 시점의 옵션 가치의 가중치를 합산하여 현재 시점에서의 옵션 가격을 얻는다.

2. **미국형 콜 옵션의 조기 실행**:
   배당금을 지급하지 않는 경우, 미국형 콜 옵션을 조기 실행하는 것이 최적이 아닌 이유는 옵션의 시간 가치 때문이다. 옵션을 만기 전 조기 실행하면, 이 옵션이 만기까지 가지고 있을 수 있는 추가적인 시간 가치를 포기하게 된다. 주식이 배당금을 지급하지 않으므로 조기 실행의 이점이 없어, 만기까지 기다려 옵션의 가치를 극대화하는 것이 최선이다.

3. **유럽형 콜 옵션과 미국형 콜 옵션의 동일성**:
   배당금을 지급하지 않는 기초 자산에 대해 유럽형 콜 옵션과 미국형 콜 옵션이 동일하다는 주장은 두 옵션의 가격이 같다는 의미이다. 배당금이 없을 경우, 두 옵션 모두 동일한 가격으로 결제되어 만기까지의 시장 조건에 따라 가치가 결정되고, 조기 실행의 이점이 없기 때문에 두 옵션의 가격 차이가 발생하지 않는다. 즉, 미국형 콜 옵션은 유럽형의 가격과 같고 추가 비용도 발생하지 않는다.

4. **

## 3. Quality Control

In [72]:
quality_control_prompt_system="""You are the final sensitivity reader for a benchmark that is about to be published.
Read through a question included in the benchmark and evaluate whether the question is answerable.
A question is deemed unanswerable if:

1. The question does not include the name of a company.
2. The question is based on information that is outdated or no longer relevant.
3. The question assumes knwledge that is not commonly available or requires specialized expertise beyond the scape of the benchmark audience.
4. The question's phrasing is ambiguous or can lead to multiple equally valid interpertations.
5. The information needed to answer the question correctly is not present within the context provided in the benchmark.
6. The question contains biases or assumptions that could disadvantage certain group of people or promote stereotypes.

please ensure that each question in the benchmark meets these criteria to be considered answerable and appropriate for publication. Explain wheter the question meets each criteria and return [[Yse]] for answerable question and [[No]] for unanswerable questions.
"""

quality_control_prompt_user = """
### Question: {}
### Gold Answer: {}
### Wrong Answers:
    1. {}
    2. {}
    3. {}
    4. {}

### Decision:
"""

In [74]:
# 0.2달러 소모
wrong_answers = filtered_selections[1:5]
qc_system_prompt = {'role':'system', 'content':quality_control_prompt_system}
qc_user_prompt = {'role':'user', 'content': quality_control_prompt_user.format(question, gold_answer, wrong_answers[0], wrong_answers[1], wrong_answers[2], wrong_answers[3])}

response = client.chat.completions.create(
        model='gpt-4-turbo',
        messages = [qc_system_prompt, qc_user_prompt]
    )

print(response.choices[0].message.content)


The question in consideration asks about the evaluation of American-style call options' pricing when the underlying stock does not pay dividends, specifically using a simplified binomial model and applying risk-neutral assumptions. This complex finance-related question also has sub-questions each pertaining to either European or American call options in different scenarios.

1. **Company Name Inclusion**: The question does not require the name of a specific company as it deals with option pricing concepts in general. Thus, criteria 1 is met.
   
2. **Outdated or Irrelevant Information**: The question is based on financial models and theoretical frameworks that are standard and currently relevant in financial studies and applications. Criteria 2 is met.

3. **Specialized Knowledge beyond Benchmark Audience**: The question delves deeply into financial modeling, including the binomial model and differences between American and European options. It demands a high level of understanding of 

# 최종 문제

In [76]:
print(quality_control_prompt_user.format(question, gold_answer, wrong_answers[0], wrong_answers[1], wrong_answers[2], wrong_answers[3]))


### Question: 주식이 배당금을 지급하지 않는 경우, 미국형 콜 옵션의 가격을 평가하는 방법에 대해 설명하시오. 구체적으로 단순한 이항모형을 사용하며, 위험 중립 가정을 적용하는 방법을 서술하시오. 다음 질문에 답하시오:

1. 유럽형 콜 옵션의 가격이 이항모형에서 어떻게 계산되는지 설명하시오.
2. 미국형 콜 옵션에 대한 조기 실행이 최적이 아닌 이유를 논하시오.
3. 배당금을 지급하지 않는 기초 자산에 대해 유럽형 콜 옵션과 미국형 콜 옵션이 동일하다고 하는 주장의 의미는 무엇인지 설명하시오.
4. 깊게 인더 머니(deep in-the-money)인 미국형 콜 옵션의 가격을 어떻게 결정할 수 있는지, 그리고 왜 옵션 가격이 (1)식 대신 (2)식으로 계산되어야 하는지를 설명하시오. 

위 질문에 대한 답변을 작성하시오.
### Gold Answer: 1. **유럽형 콜 옵션의 가격 계산**:
   이항모형에서 유럽형 콜 옵션의 가격은 주식 가격이 여러 시점에 걸쳐 두 가지 가능한 값 중 하나로 변할 수 있는 구조를 기반으로 한다. 이항 구조에서는 특정 시간 t에 주가가 상승할 확률 p와 하락할 확률 (1-p)을 설정하고, 각 기초 자산의 가격 변수를 다음과 같이 정의한다:
   - S_up = S * u (주가 상승)
   - S_down = S * d (주가 하락)
   여기서 u는 상승 비율, d는 하락 비율이다. 옵션의 만기 시점에서의 가치 C(T) = max(0, S(T) - K)로 정의되며, 이 값을 기초 자산의 변화에 따라 이산적으로 계산한다. 위험 중립적 확률을 사용해 현재 가치를 계산하고, 최종 시점의 옵션 가치의 가중치를 합산하여 현재 시점에서의 옵션 가격을 얻는다.

2. **미국형 콜 옵션의 조기 실행**:
   배당금을 지급하지 않는 경우, 미국형 콜 옵션을 조기 실행하는 것이 최적이 아닌 이유는 옵션의 시간 가치 때문이다. 옵션을 만기 전 조기 실행하면, 이 옵션이 만기까지 가지고 있을 수 있는 추가적인 시간 가치를 포기하